In [1]:
import torch
import torch.nn as nn
from torchvision.models import inception_v3
from torch.optim import SGD, Adam
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, Subset
from PIL import Image
import os

/Users/muhammadwaseem/miniconda3/envs/torch/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/Users/muhammadwaseem/miniconda3/envs/torch/lib/python3.9/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefIxEENS2_8optionalINS2_10ScalarTypeEEENS5_INS2_6LayoutEEENS5_INS2_6DeviceEEENS5_IbEENS5_INS2_12MemoryFormatEEE
  Referenced from: <CAE66874-17C2-35C9-9C4D-6BA9770AF17F> /Users/muhammadwaseem/miniconda3/envs/torch/lib/python3.9/site-packages/torchvision/image.so
  Expected in:     <459875AA-DE2C-366B-9C44-90D4B3887080> /Users/muhammadwaseem/miniconda3/envs/torch/lib/python3.9/site-packages/torch/lib/libtorch_cpu.dylib
  warn(f"Failed to load image Python extension: {e}")


In [2]:
transform = transforms.Compose([
                                transforms.Resize([342], interpolation=transforms.InterpolationMode.BILINEAR),
                                transforms.CenterCrop([299,299]),
                                transforms.ToTensor(), # Rescaled to 0-1 aswell
                                transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)) # Mean and std of imageNet
])

target_transform = transforms.Compose([
                                transforms.Resize([342], interpolation=transforms.InterpolationMode.BILINEAR),
                                transforms.CenterCrop([299,299]),
                                transforms.ToTensor(),
])

In [26]:
class CustomDataset(Dataset):
    def __init__(self, path, transform=None, target_transform=None):
        self.transform= transform
        self.target_transform = target_transform
        self.path = path
        self.count = 0
        for i in os.listdir(self.path)[1:]: # [1:] because MacOS has .DS_Store at 0th index
            for j in os.listdir(os.path.join(self.path,i)):
                self.count += 1
        
    def __len__(self):
        return self.count
    
    def __getitem__(self, idx):
        # Lazy loading
        
        directory_idx = idx//100
        file_idx = idx%100
        
        label = directory_idx
        
        directory = os.listdir(self.path)[directory_idx + 1]
        flower_directory = os.path.join(self.path, directory)
        image_path = os.path.join(flower_directory, os.listdir(flower_directory)[file_idx])
    
        image = Image.open(image_path)
        image = target_transform(image)

        return image, label
        

In [27]:
dataset = CustomDataset('tiny_FR', target_transform=target_transform)

In [28]:
torch.manual_seed(10)
index_train_test = torch.randperm(500)
index_train_val = torch.randperm(400)
data = Subset(dataset, index_train_test[:400])
data_test = Subset(dataset, index_train_test[400:])
data_train = Subset(data, index_train_val[:300])
data_valid = Subset(data, index_train_val[300:])

In [29]:
torch.manual_seed(1)
data = DataLoader(data_train, 32, shuffle=True)
data_vl = DataLoader(data_valid, 16, shuffle=True)

In [30]:
next(iter(data))[0].shape

torch.Size([32, 3, 299, 299])

In [31]:
model = inception_v3(weights="IMAGENET1K_V1")

In [32]:
model

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

In [33]:
for param in model.parameters():
    param.requires_grad = False

In [34]:
num_ftrs = model.AuxLogits.fc.in_features
model.AuxLogits.fc = nn.Linear(num_ftrs, 5)

num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 5)

In [35]:
params_to_update = []
for name,param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print(param.shape)
        print("\t",name)
        


torch.Size([5, 768])
	 AuxLogits.fc.weight
torch.Size([5])
	 AuxLogits.fc.bias
torch.Size([5, 2048])
	 fc.weight
torch.Size([5])
	 fc.bias


In [36]:
for i in model.fc.parameters():
    print(i.shape)

torch.Size([5, 2048])
torch.Size([5])


In [37]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params_to_update, lr=0.03, momentum=0.9)

In [38]:
def train(epoch, data, data_vl, PATH="model_inception.pt"):
    train_loss = [0] * epoch
    train_accuracy = [0] * epoch
    
    valid_loss = [0] * epoch
    valid_accuracy = [0] * epoch
    low_val_loss = 1000
    #model.train()
    for i in range(epoch):
        for x_batch, y_batch in data:
            #print(x_batch.shape)
            
            pred, _ = model(x_batch)
            loss = loss_fn(pred, y_batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            
            train_loss[i] += loss.item() * x_batch.size(0)
            correct = torch.sum(torch.argmax(torch.softmax(pred,axis=1), axis=1) == y_batch)
            train_accuracy[i] += correct.to(torch.float32).item()
        train_loss[i] /= len(data.dataset)
        train_accuracy[i] /= len(data.dataset)
        
        with torch.no_grad():
            for x_batch, y_batch in data_vl:
                pred, _ = model(x_batch)
                loss = loss_fn(pred, y_batch)
                valid_loss[i] += loss.item() * x_batch.size(0)
                correct = torch.sum(torch.argmax(torch.softmax(pred,axis=1), axis=1) == y_batch)
                valid_accuracy[i] += correct.item()
            valid_loss[i] /= len(data_vl.dataset)
            valid_accuracy[i] /= len(data_vl.dataset)
        if valid_loss[i] < low_val_loss:
            torch.save({
            'epoch': i +1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': valid_loss[i],
            }, PATH)
            low_val_loss = valid_loss[i]
        print(f'Epoch {i+1} accuracy: {train_accuracy[i]:.4f} val_accuracy:{valid_accuracy[i]:.4f}')
        print(f'Epoch {i+1} loss: {train_loss[i]:.4f} val_loss:{valid_loss[i]:.4f}')
        print()
            

In [39]:
train(30, data, data_vl)

Epoch 1 accuracy: 0.4267 val_accuracy:0.5600
Epoch 1 loss: 1.3346 val_loss:1.0411

Epoch 2 accuracy: 0.7233 val_accuracy:0.7500
Epoch 2 loss: 0.7933 val_loss:0.7357

Epoch 3 accuracy: 0.7800 val_accuracy:0.7100
Epoch 3 loss: 0.6601 val_loss:0.9790

Epoch 4 accuracy: 0.8100 val_accuracy:0.7300
Epoch 4 loss: 0.5176 val_loss:0.8302

Epoch 5 accuracy: 0.8867 val_accuracy:0.7400
Epoch 5 loss: 0.3401 val_loss:1.1112

Epoch 6 accuracy: 0.8633 val_accuracy:0.7900
Epoch 6 loss: 0.4069 val_loss:0.6461

Epoch 7 accuracy: 0.8900 val_accuracy:0.6900
Epoch 7 loss: 0.3386 val_loss:0.8527

Epoch 8 accuracy: 0.9233 val_accuracy:0.8100
Epoch 8 loss: 0.2845 val_loss:0.5900

Epoch 9 accuracy: 0.9233 val_accuracy:0.7700
Epoch 9 loss: 0.2583 val_loss:0.7803

Epoch 10 accuracy: 0.8800 val_accuracy:0.7800
Epoch 10 loss: 0.2663 val_loss:0.8029

Epoch 11 accuracy: 0.9367 val_accuracy:0.7200
Epoch 11 loss: 0.1902 val_loss:0.9851

Epoch 12 accuracy: 0.9333 val_accuracy:0.8500
Epoch 12 loss: 0.1772 val_loss:0.5571

In [40]:
checkpoint = torch.load("model_inception.pt")
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']
epoch, loss

(16, 0.4952272665500641)

In [43]:
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix

In [48]:
data_tst = DataLoader(data_test, 100, False)
test_data = next(iter(data_tst))[0]
y_pred,_ = model(test_data)
# print(y_pred.shape)
y_pred = torch.argmax(torch.softmax(y_pred, axis=1),axis=1)
y_test = next(iter(data_tst))[1]

In [49]:
# y_pred = torch.detach(y_pred.to('cpu')).numpy()
# y_test = torch.detach(y_test.to('cpu')).numpy()

In [50]:
p = precision_score(y_test, y_pred, average = 'weighted')
print(f"Precision: {p}")

r = recall_score(y_test, y_pred , average = 'weighted')
print(f"Recall: {r}")

f1 = f1_score( y_test, y_pred, average = 'weighted')
print(f"F1-score: {f1}")

Precision: 0.7170281892021022
Recall: 0.7
F1-score: 0.6887976037655114
